<a href="https://colab.research.google.com/github/nhwzaan/multimedia_computing/blob/main/20520855_BT8_PCA_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Thông tin sinh viên



- Họ tên: Nguyễn Thị Như Vân
- MSSV: 20520855

# Import thư viện

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import math
import os
import time

In [ ]:
pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
import soundfile as sf

# PCA

In [ ]:
def ChoosingK(S):
  k=S.shape[0]+1
  variance_is_remained=1
  while variance_is_remained>=0.96:
    k-=1
    variance_is_remained=S[:k].sum(axis=0)/S.sum(axis=0)
  return k+1

## Encode

In [ ]:
def PCA(X, K = 1):
  X = X.T
  M = X.shape[1]
  X_mean = X.mean(axis=1, keepdims=True)
  X_hat_n = X - X_mean
  Sigma = np.dot(X_hat_n, X_hat_n.T)/M
  U, S, _ = np.linalg.svd(Sigma, full_matrices=True)
  U_k = U[:,:K]
  Z_k = np.dot(U_k.T, X_hat_n)
  return U_k, Z_k, X_mean

## Tính toán lượng thông tin được giữ lại

In [ ]:
def calculateRk(X, K):
  X = X.T
  M = X.shape[1]
  X_mean = X.mean(axis=1,keepdims=True)
  X_ = X-X_mean
  Sigma = np.dot(X_,X_.T)/M
  _, S, _ = np.linalg.svd(Sigma, full_matrices=True)

  return S[:K].sum(axis=0)/S.sum(axis=0)

## Decode

In [ ]:
def decodePCA(U_k, Z_k, X_mean):
  return (U_k.dot(Z_k)+X_mean).T

# Nén âm thanh với PCA

## Load file âm thanh

In [ ]:
! wget 'https://github.com/trivm12/test/raw/main/Mondstadt%20at%20Night%20-%20Genshin%20Impact%20(Piano%20cover%20-%20Pianuki).mp3'

--2023-04-03 15:00:04--  https://github.com/trivm12/test/raw/main/Mondstadt%20at%20Night%20-%20Genshin%20Impact%20(Piano%20cover%20-%20Pianuki).mp3
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/trivm12/test/main/Mondstadt%20at%20Night%20-%20Genshin%20Impact%20(Piano%20cover%20-%20Pianuki).mp3 [following]
--2023-04-03 15:00:04--  https://raw.githubusercontent.com/trivm12/test/main/Mondstadt%20at%20Night%20-%20Genshin%20Impact%20(Piano%20cover%20-%20Pianuki).mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 828021 (809K) [audio/mpeg]
Saving to: ‘Mondstadt at Night - Genshin Impact (Piano cover

In [ ]:
! mv "/content/Mondstadt at Night - Genshin Impact (Piano cover - Pianuki).mp3" "/content/genshin.mp3"

## PCA

In [ ]:
def PCA_encode_audio(filePath, K = 1):
  import librosa

  y, sr = librosa.load(filePath)
  exponentiation = round(math.sqrt(len(y)))
  Shape = (exponentiation, exponentiation - 1)
  padding_adding = 0
  if Shape[0] * Shape[1] >= len(y):
    padding_adding = Shape[0] * Shape[1] - len(y)
  else:
    padding_adding = Shape[0] * (Shape[1] + 1) - len(y)
    Shape = (exponentiation, exponentiation)

  encoded = [sr, len(y)]
  y = np.append(y, np.array([0.] * padding_adding, dtype=np.float32))
  y = y.reshape(Shape)
  U_k, Z_k, X_mean = PCA(y, K)
  encoded += [U_k, Z_k, X_mean]

  return encoded

In [ ]:
def PCA_decode_audio(encoded):
  # STRUCTURE OF ENCODED
  #   encoded = [
  #                 sampling rate,
  #                 Original_length,
  #                 U_k,
  #                 Z_k,
  #                 X_mean
  #             ]

  decoded = decodePCA(encoded[2], encoded[3], encoded[4])
  decoded = decoded.flatten()[:encoded[1]]
  sr = encoded[0]
  return sr, decoded

## Test

In [ ]:
from IPython.display import Audio, display
print("Original sound:")
display(Audio("/content/genshin.mp3", autoplay=True))

Original sound:


In [ ]:
audio_PCA = PCA_encode_audio('/content/genshin.mp3', K=9)

In [ ]:
sr, audio_PCA_decoded = PCA_decode_audio(audio_PCA)

In [ ]:
sf.write('/content/decoded_genshin.wav', audio_PCA_decoded, sr)

In [ ]:
import pydub
sound = pydub.AudioSegment.from_wav("/content/decoded_genshin.wav")
sound.export("/content/decoded_genshin.mp3", format="mp3")

<_io.BufferedRandom name='/content/decoded_genshin.mp3'>

In [ ]:
print(" Original file:",
      os.path.getsize("/content/genshin.mp3"))
print("Decoded .wav file:",
      os.path.getsize("/content/decoded_genshin.wav"))
print("Decoded .mp3 file:",
      os.path.getsize("/content/decoded_genshin.mp3"))

 Original file: 828021
Decoded .wav file: 2279900
Decoded .mp3 file: 207326
